In [1]:
import pandas as pd

In [24]:
# files contains human annotated data as ground truth
annotation_dir = "human_annotation.csv"
annotated_data = pd.read_csv(annotation_dir)

# Filter out meaningless responses

In [27]:
meaningless_lst = ['goodbye', 'hi', 'yes', 'ok', 'lol', 'hello', 'goodbye', 'good bye', 'thanks', 'thank you']
                

In [28]:
df = annotated_data

In [29]:
# 1. Drop 'Unnamed: 0' column
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# 2. Drop rows based on 'recommend_sentence' condition
def filter_sentences(sentence):
    for word in meaningless_lst:
        sentence = sentence.lower().replace(word, '')
    return len(sentence.strip()) >= 10

#df = df[df['recommend_sentence'].apply(filter_sentences)]


In [30]:
clean_df = df[df['annotation'] == 1].copy()

# drop annotation column
clean_df.drop(columns=['annotation', 'type'], inplace=True)

In [31]:
df = clean_df

# create eval samples

In [32]:
import pandas as pd
import itertools
import json
import random


In [33]:

# Sample dataframe
# df = pd.DataFrame({
#     'recommend_sentence': ['sentence1', 'sentence2', 'sentence3', ...],
#     'labels': [0, 1, 2, ...]
# })

# Step 1: Get positive samples
positive_samples = []
unique_labels = df['labels'].unique()

for label in unique_labels:
    filtered_df = df[df['labels'] == label]
    # Create all possible combinations of two rows
    pairs = list(itertools.combinations(filtered_df['recommend_sentence'], 2))
    positive_samples.extend(pairs)

# Step 2: Convert positive samples to required format
positive_samples_json = [{'text': f"{pair[0]} [SEP] {pair[1]}", 'label': 1} for pair in positive_samples]

# Step 3: Create negative samples
negative_samples = []

while len(negative_samples) < len(positive_samples):
    row1, row2 = df.sample(2).values
    if row1[1] != row2[1]:  # Ensure they have different labels
        negative_samples.append((row1[0], row2[0]))

# Step 4: Convert negative samples to required format
negative_samples_json = [{'text': f"{pair[0]} [SEP] {pair[1]}", 'label': 0} for pair in negative_samples]

# Combine positive and negative samples
final_dataset = positive_samples_json + negative_samples_json

# Shuffle the dataset to mix positive and negative samples
random.shuffle(final_dataset)




In [34]:
# Save the dataset
with open("redial_eval_dataset.json", 'w') as f:
    json.dump(final_dataset, f)
print("sample numbers = ", len(final_dataset))

42006
